In [1]:
%%capture
!pip install -U crowd-kit==1.0.0

In [2]:
import pandas as pd

In [3]:
# X. Chen, P.N. Bennett, K. Collins-Thompson, E. Horvitz. Pairwise Ranking Aggregation in a Crowdsourced Setting. Proceedings of WSDM 2013. 193-202
df = pd.read_csv('http://www-personal.umich.edu/~kevynct/datasets/wsdm_rankagg_2013_readability_crowdflower_data.csv', dtype=str)
df

,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_trust,_worker_id,please_decide_which_passage_is_more_difficult,label_level_a,label_level_b,passage_a,passage_b,please_decide_which_passage_is_more_difficult_gold
0,101706900,9/22/2011 10:18,FALSE,213212730,NaN,9/22/2011 9:55,TRUE,0.3333,1848539,Passage B is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
1,101706900,9/22/2011 13:18,FALSE,213286745,NaN,9/22/2011 13:15,FALSE,0.8571,1180070,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
2,101706900,9/22/2011 13:42,FALSE,213298361,NaN,9/22/2011 13:41,FALSE,0.8571,3888076,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
3,101706900,9/22/2011 15:05,FALSE,213338956,NaN,9/22/2011 15:03,TRUE,1,3674807,Passage B is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
4,101706900,9/22/2011 16:39,FALSE,213387820,NaN,9/22/2011 16:39,TRUE,0.5,756883,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13851,101708371,9/23/2011 9:34,FALSE,213820869,NaN,9/23/2011 9:32,FALSE,1,1147266,Passage B is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13852,101708371,9/24/2011 1:37,FALSE,214258697,NaN,9/24/2011 1:36,FALSE,1,4421031,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13853,101708371,9/25/2011 5:30,FALSE,215015312,NaN,9/25/2011 5:27,TRUE,0.6667,4066097,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13854,101708371,9/25/2011 6:00,FALSE,215026304,NaN,9/25/2011 5:57,FALSE,1,475751,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN


In [4]:
for column, status in (('passage_a', 'Passage A is more difficult.'), ('passage_b', 'Passage B is more difficult.')):
    df.loc[df['please_decide_which_passage_is_more_difficult'] == status, 'label'] = df.loc[df['please_decide_which_passage_is_more_difficult'] == status, column]

In [5]:
df.rename(columns={
    '_worker_id': 'worker',
    'passage_a': 'left',
    'passage_b': 'right'
}, inplace=True)

df.dropna(subset=['label'], inplace=True)

In [6]:
df[['worker', 'left', 'right', 'label']]

,worker,left,right,label
0,1848539,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,do it yourself by reading the directions When ...
1,1180070,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
2,3888076,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
3,3674807,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,do it yourself by reading the directions When ...
4,756883,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
...,...,...,...,...
13851,1147266,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,the frequent conflict between flood control an...
13852,4421031,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."
13853,4066097,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."
13854,475751,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."


In [7]:
from crowdkit.aggregation import BradleyTerry, NoisyBradleyTerry

In [8]:
agg_bt = BradleyTerry(n_iter=100).fit_predict(df)

In [9]:
agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(df)

In [10]:
gt_levels = {}

for _, row in df.iterrows():
    gt_levels[row['left']] = int(row['label_level_a'])
    gt_levels[row['right']] = int(row['label_level_b'])

gt = pd.Series(gt_levels.values(), index=gt_levels.keys())

In [11]:
df_agg = pd.DataFrame({'bt': agg_bt, 'noisybt': agg_noisybt, 'gt': gt}).reset_index()
df_agg.rename(columns={'index': 'passage'}, inplace=True)
df_agg

,passage,bt,noisybt,gt
0,'Arc Vallon Pont 'Arc One interesting thing yo...,0.002944,0.257941,6
1,. . Main engine ignition sequence started . . ...,0.002213,0.992986,4
2,. and to explore the outdoor world. The packs ...,0.001457,0.012690,11
3,". early invading species like horseweed, aster...",0.004883,0.999994,11
4,Accords. The Accords were approved in May by p...,0.003649,0.999999,6
...,...,...,...,...
485,"yellow, what is the probability that the next ...",0.001734,0.009689,11
486,you are real. I hope you are not just in my im...,0.000556,0.876347,2
487,you define cyberspace The information in this ...,0.005225,0.983429,11
488,you from getting in debt again. But if I shoul...,0.002131,0.692790,7


In [12]:
import numpy as np

np.random.seed(0)

df_agg['bt_rank'] = df_agg['bt'].rank().astype(int)
df_agg['noisybt_rank'] = df_agg['noisybt'].rank().astype(int)
df_agg['random_rank'] = np.random.permutation(range(len(df_agg)))

df_agg

,passage,bt,noisybt,gt,bt_rank,noisybt_rank,random_rank
0,'Arc Vallon Pont 'Arc One interesting thing yo...,0.002944,0.257941,6,369,205,238
1,. . Main engine ignition sequence started . . ...,0.002213,0.992986,4,293,366,179
2,. and to explore the outdoor world. The packs ...,0.001457,0.012690,11,203,131,437
3,". early invading species like horseweed, aster...",0.004883,0.999994,11,469,462,325
4,Accords. The Accords were approved in May by p...,0.003649,0.999999,6,410,472,15
...,...,...,...,...,...,...,...
485,"yellow, what is the probability that the next ...",0.001734,0.009689,11,231,119,323
486,you are real. I hope you are not just in my im...,0.000556,0.876347,2,83,279,192
487,you define cyberspace The information in this ...,0.005225,0.983429,11,478,340,117
488,you from getting in debt again. But if I shoul...,0.002131,0.692790,7,283,248,47


In [13]:
from sklearn.metrics import ndcg_score

In [14]:
ndcg_score(df_agg['random_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.43069374799178833

In [15]:
ndcg_score(df_agg['bt_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.7104760157742371

In [16]:
ndcg_score(df_agg['noisybt_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.7464401942586768

In [17]:
df_agg[['bt_rank', 'noisybt_rank', 'random_rank', 'gt']].corr()

,bt_rank,noisybt_rank,random_rank,gt
bt_rank,1.000000,0.422565,0.030279,0.423757
noisybt_rank,0.422565,1.000000,0.011692,0.228200
random_rank,0.030279,0.011692,1.000000,-0.043134
gt,0.423757,0.228200,-0.043134,1.000000


In [18]:
df_agg.sort_values('bt_rank', ascending=False, inplace=True)
df_agg

,passage,bt,noisybt,gt,bt_rank,noisybt_rank,random_rank
305,"my grief, but I cannotLove is too much for me....",0.006496,0.999081,8,490,405,487
46,Sun ArtCodes REVE.ARI never knew a mountaintop...,0.006412,0.999968,8,489,448,118
9,D. Habeas corpus Which Supreme Court case over...,0.006372,1.000000,11,488,478,459
433,"this month. A pleasant walk to school, just on...",0.006258,0.996739,4,487,382,370
420,the sky Five nights after the new moon in Janu...,0.006065,0.725967,3,486,256,388
...,...,...,...,...,...,...,...
55,This is a teacher. He is an important helper. ...,0.000000,0.156456,1,15,194,391
389,"tears in her eyes. You look sad, said Sam. I a...",0.000000,0.598311,3,15,237,148
44,Some people live deep below the water. Some pe...,0.000000,0.660779,1,15,245,78
252,in the pond. They breathe under the water. Dow...,0.000000,0.682509,1,15,246,46
